In [23]:
import re
from google.colab import auth
from googleapiclient.discovery import build
from google.auth import default
from googleapiclient.errors import HttpError


In [24]:
def authenticate_docs_api():
    try:
        auth.authenticate_user()
        creds, _ = default(scopes=[
            "https://www.googleapis.com/auth/documents",
            "https://www.googleapis.com/auth/drive"
        ])
        return build("docs", "v1", credentials=creds)
    except Exception as e:
        raise RuntimeError(f"Authentication failed: {e}")


In [25]:
docs_service = authenticate_docs_api()


In [26]:
def create_document(service, title):
    try:
        doc = service.documents().create(
            body={"title": title}
        ).execute()
        return doc["documentId"]
    except HttpError as e:
        raise RuntimeError(f"Failed to create document: {e}")


In [31]:
def clear_document(service, document_id):
    try:
        doc = service.documents().get(documentId=document_id).execute()
        body = doc.get("body", {}).get("content", [])

        if len(body) > 1:
            end_index = body[-1]["endIndex"]
            service.documents().batchUpdate(
                documentId=document_id,
                body={
                    "requests": [
                        {
                            "deleteContentRange": {
                                "range": {
                                    "startIndex": 1,
                                    "endIndex": end_index - 1
                                }
                            }
                        }
                    ]
                }
            ).execute()
    except HttpError as e:
        raise RuntimeError(f"Failed to clear document: {e}")


In [32]:
DOCUMENT_ID = create_document(
    docs_service, "Product Team Sync – May 15, 2023"
)
print("Created document:", DOCUMENT_ID)


Created document: 1xd4kefDZheftqT29dftShxfk7yKmAeMgpe8h22-vWvs


In [33]:
MARKDOWN_TEXT = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [34]:
def bullet_level(line):
    spaces = len(line) - len(line.lstrip(" "))
    return spaces // 2


In [35]:
def insert_text_request(text, index):
    return {
        "insertText": {
            "location": {"index": index},
            "text": text
        }
    }


In [36]:
def build_requests_from_markdown(markdown):
    requests = []
    cursor = 1  # Google Docs body starts at index 1

    for raw_line in markdown.split("\n"):
        line = raw_line.rstrip()

        if not line.strip():
            continue

        try:
            # H1
            if line.startswith("# "):
                text = line[2:] + "\n"
                requests.append(insert_text_request(text, cursor))
                requests.append({
                    "updateParagraphStyle": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                        "paragraphStyle": {"namedStyleType": "HEADING_1"},
                        "fields": "namedStyleType"
                    }
                })
                cursor += len(text)

            # H2
            elif line.startswith("## "):
                text = line[3:] + "\n"
                requests.append(insert_text_request(text, cursor))
                requests.append({
                    "updateParagraphStyle": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                        "paragraphStyle": {"namedStyleType": "HEADING_2"},
                        "fields": "namedStyleType"
                    }
                })
                cursor += len(text)

            # H3
            elif line.startswith("### "):
                text = line[4:] + "\n"
                requests.append(insert_text_request(text, cursor))
                requests.append({
                    "updateParagraphStyle": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                        "paragraphStyle": {"namedStyleType": "HEADING_3"},
                        "fields": "namedStyleType"
                    }
                })
                cursor += len(text)

            # Checkboxes + @mentions (styled)
            elif line.strip().startswith("- [ ]"):
                content = line.strip()[5:].strip() + "\n"
                requests.append(insert_text_request(content, cursor))
                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(content)},
                        "bulletPreset": "BULLET_CHECKBOX"
                    }
                })

                # Style @mentions (bold + blue)
                mention = re.search(r"@\w+", content)
                if mention:
                    requests.append({
                        "updateTextStyle": {
                            "range": {
                                "startIndex": cursor + mention.start(),
                                "endIndex": cursor + mention.end()
                            },
                            "textStyle": {
                                "bold": True,
                                "foregroundColor": {
                                    "color": {
                                        "rgbColor": {
                                            "red": 0.11,
                                            "green": 0.33,
                                            "blue": 0.80
                                        }
                                    }
                                }
                            },
                            "fields": "bold,foregroundColor"
                        }
                    })

                cursor += len(content)

            # Nested bullets
            elif line.lstrip().startswith(("-", "*")):
                level = bullet_level(line)
                content = line.lstrip()[1:].strip() + "\n"
                requests.append(insert_text_request(content, cursor))
                requests.append({
                    "createParagraphBullets": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(content)},
                        "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                    }
                })

                if level > 0:
                    requests.append({
                        "updateParagraphStyle": {
                            "range": {"startIndex": cursor, "endIndex": cursor + len(content)},
                            "paragraphStyle": {
                                "indentStart": {
                                    "magnitude": level * 18,
                                    "unit": "PT"
                                }
                            },
                            "fields": "indentStart"
                        }
                    })

                cursor += len(content)

            # Footer (distinct style)
            elif line.startswith("Meeting recorded") or line.startswith("Duration"):
                text = line + "\n"
                requests.append(insert_text_request(text, cursor))
                requests.append({
                    "updateTextStyle": {
                        "range": {"startIndex": cursor, "endIndex": cursor + len(text)},
                        "textStyle": {
                            "italic": True,
                            "fontSize": {
                                "magnitude": 9,
                                "unit": "PT"
                            },
                            "foregroundColor": {
                                "color": {
                                    "rgbColor": {
                                        "red": 0.45,
                                        "green": 0.45,
                                        "blue": 0.45
                                    }
                                }
                            }
                        },
                        "fields": "italic,fontSize,foregroundColor"
                    }
                })
                cursor += len(text)

            # Default text
            else:
                text = line + "\n"
                requests.append(insert_text_request(text, cursor))
                cursor += len(text)

        except Exception as e:
            raise RuntimeError(f"Failed parsing line: '{line}' → {e}")

    return requests


In [37]:
def apply_updates(service, document_id, requests):
    try:
        service.documents().batchUpdate(
            documentId=document_id,
            body={"requests": requests}
        ).execute()
    except HttpError as e:
        raise RuntimeError(f"Document update failed: {e}")


In [38]:
requests = build_requests_from_markdown(MARKDOWN_TEXT)
apply_updates(docs_service, DOCUMENT_ID, requests)
print("Document successfully formatted.")


Document successfully formatted.
